匹配AVHRR信息和轨迹信息

In [17]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import matplotlib.patches as patches
import cmaps
import cartopy.feature as cfeature
from datetime import datetime,timedelta
from matplotlib.offsetbox import AnchoredText#关于修改text位置的
from datetime import datetime,timedelta
import cv2
from pygac import get_reader_class
import ast
from tqdm import tqdm
import warnings
from sklearn.neighbors import BallTree
from shapely.geometry import Point, Polygon,LineString
from shapely.ops import nearest_points
import json
import shutil
import warnings
warnings.filterwarnings("ignore")

In [2]:
def geodistance(lon1,lat1,lon2,lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2]) # 经纬度转换成弧度
    dlon=lon2-lon1
    dlat=lat2-lat1
    a=np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    distance=2*np.arcsin(np.sqrt(a))*6371*1000 # 地球平均半径，6371km
    distance=np.round(distance,3)
    return distance/1000
def xy2lonlat(chosed_lon,chosed_lat,x,y):
    #距离转经纬度
    #纬圈距离
    earth_radius = 6371.0
    lat_circum = np.cos(np.radians(chosed_lat))*2*np.pi*earth_radius
    d_lon = x/lat_circum*360
    d_lat = y/(2*np.pi*earth_radius)*360
    new_lon,new_lat = chosed_lon+d_lon,chosed_lat+d_lat
    return new_lon,new_lat
#将经纬度转换为极射投影,并求极坐标系中的最近距离和最近点，注意该最近距离指极坐标距离而非球面距离
def nearest_judge(chosed_lon,chosed_lat,border):
    #似乎只能一对一
    #1、计算平面上各点到极点的距离和夹角
    earth_radius = 6371.0
    r = (90-border[:,1])/90*earth_radius
    theta = np.radians(border[:,0])
    #2、将边界转换为x,y坐标，注意该坐标系以-90°为y轴负轴，相当于中心经度设置为-90
    x,y = np.cos(theta)*r,np.sin(theta)*r
    points_coords = np.concatenate([np.reshape(x,(-1,1)),np.reshape(y,(-1,1))],axis=1) 
    #将中心点转换为x,y
    r0 = (90-chosed_lat)/90*earth_radius
    theta0 = np.radians(chosed_lon)
    x0,y0 = np.cos(theta0)*r0,np.sin(theta0)*r0
    #3、计算最近点
    point = Point(x0,y0)
    Line_border = LineString(points_coords)
    #4、判断点是否在多边形内,这里判断是否在内要用polygon对象
    inside = point.within(Polygon(points_coords))
    nearest_point,_= nearest_points(Line_border, point)
    near_x,near_y = nearest_point.coords[0]
    min_dis = np.sqrt((near_x-x0)**2+(near_y-y0)**2)
    #print(inside,min_dis)
    near_r = np.sqrt(near_x**2+near_y**2)
    #反演回经纬度
    near_lon = np.rad2deg(np.arctan2(near_y ,near_x))
    near_lat = (1-near_r/earth_radius)*90
    return near_lon,near_lat,inside,min_dis
#计算点之间距离
def deal_distance(line):
    distance = [0]
    for i in range(len(line)-1):
        distance.append(np.round(geodistance(line[i][0],line[i][1],line[i+1][0],line[i+1][1]),3))
    return distance

In [3]:
AVHRR_info = pd.read_excel(r"D:\Desktop\pycode\PLs_identity\IMPMCT\example_data\AVHRR_infos_2008_11.xlsx")
AVHRR_info['border'] = AVHRR_info['border'].apply(lambda x :np.array(ast.literal_eval(x)).T)
ERA5_tracks = pd.read_excel(r"D:\Desktop\pycode\PLs_identity\IMPMCT\example_data\track_2008_11.xlsx",usecols=['time','path_index','lon','lat','peak_value'])
ERA5_tracks

,time,path_index,lon,lat,peak_value
0,2008-11-01 00:00:00,0,-18.25,72.25,1.576954
1,2008-11-01 00:00:00,1,-0.50,77.00,1.815238
2,2008-11-01 00:00:00,2,-30.25,66.00,2.566751
3,2008-11-01 00:00:00,3,27.00,64.50,1.591397
4,2008-11-01 00:00:00,4,-9.50,72.50,1.675173
...,...,...,...,...,...
10149,2008-11-30 23:00:00,938,29.25,66.75,1.307585
10150,2008-11-30 22:00:00,939,46.25,60.75,1.313290
10151,2008-11-30 23:00:00,939,46.75,60.50,1.320050
10152,2008-11-30 23:00:00,940,-15.75,65.75,1.524033


In [4]:
ERA5_tracks['split_path_index'] = ERA5_tracks['path_index'].values
#分割距离较长的气旋路径
threshold = 200#KM,当距离超过200KM时且涡度小于1.5时，认为路径断开，这种情况常常因为涡旋区域的中心发生了改变，这样处理认为改变后的涡旋为新生涡旋，原涡旋为衰退涡旋
path_reindex = ERA5_tracks.path_index.max()+1
new_track_record = []
for path_index in tqdm(ERA5_tracks.path_index.unique()):
    pick_track = ERA5_tracks.loc[ERA5_tracks.path_index==path_index]
    #计算轨迹中点与点之间的距离
    dis_interval = np.array(deal_distance(pick_track.loc[:,['lon','lat']].values))
    #找到分割点
    track_seg = np.concatenate((np.array([0]),np.where(dis_interval>threshold)[0],np.array([len(pick_track)])), axis=0)
    #分割不同部分
    for i in range(len(track_seg)-1):
        #第一段按照原来的Path_index
        temp_track = pick_track[track_seg[i]:track_seg[i+1]].copy()
        if (i>0)&(temp_track.iloc[0].peak_value<1.8):
            temp_track.split_path_index= path_reindex
            path_reindex += 1
        new_track_record.append(temp_track)      
#去除强度分割得到的中间path_index
ERA5_tracks = pd.concat(new_track_record,axis=0).drop(columns=['path_index']).rename(columns={'split_path_index':'path_index'})
#选择六个时次以上的轨迹
life_count = ERA5_tracks.value_counts('path_index').reset_index()
ERA5_tracks = ERA5_tracks.loc[ERA5_tracks.path_index.isin(life_count.loc[life_count['count']>5,'path_index'])].copy()

100%|██████████| 942/942 [00:00<00:00, 946.84it/s] 


In [5]:
ERA5_tracks

,time,lon,lat,peak_value,path_index
41,2008-11-01 04:00:00,-37.25,66.00,2.193079,14
42,2008-11-01 05:00:00,-34.75,65.75,3.077280,14
43,2008-11-01 06:00:00,-34.75,65.75,3.145358,14
44,2008-11-01 07:00:00,-34.75,65.75,2.937778,14
45,2008-11-01 08:00:00,-34.75,65.75,2.486365,14
...,...,...,...,...,...
10133,2008-11-30 19:00:00,-19.50,66.50,1.855620,934
10134,2008-11-30 20:00:00,-20.25,65.75,1.747692,934
10135,2008-11-30 21:00:00,-20.00,65.50,1.689195,934
10136,2008-11-30 22:00:00,-19.75,65.25,1.537685,934


In [6]:
AVHRR_info#mean_time:所有50°N以上的平均扫描时间,border:扫描图像的边界四点

,Unnamed: 0,filename,dataset_type,satellite_name,start_time,end_time,filepath,tle_name,year_month,border,mean_time
0,0,NSS.GHRR.NM.D08306.S0013.E0142.B3302829.WI,GHRR,NOAA-17,2008-11-01 00:13:00,2008-11-01 01:42:00,Z:\NOAA_AVHRR\NSS.GHRR.NM.D08306.S0013.E0142.B...,NOAA-17_tle.txt,2008-11-01,"[[-27.66072683885269, 55.25605275439017], [-67...",2008-11-01 00:18:00
1,1,NSS.GHRR.NK.D08306.S0039.E0224.B5442930.GC,GHRR,NOAA-15,2008-11-01 00:39:00,2008-11-01 02:24:00,Z:\NOAA_AVHRR\NSS.GHRR.NK.D08306.S0039.E0224.B...,NOAA-15_tle.txt,2008-11-01,"[[-55.889184064012284, 85.42360983205805], [15...",2008-11-01 01:04:00
2,2,NSS.GHRR.NN.D08306.S0057.E0251.B1777778.SV,GHRR,NOAA-18,2008-11-01 00:57:00,2008-11-01 02:51:00,Z:\NOAA_AVHRR\NSS.GHRR.NN.D08306.S0057.E0251.B...,NOAA-18_tle.txt,2008-11-01,"[[-105.29832618105682, 84.60716567687085], [10...",2008-11-01 01:21:00
3,3,NSS.GHRR.NL.D08306.S0135.E0321.B4181011.GC,GHRR,NOAA-16,2008-11-01 01:35:00,2008-11-01 03:21:00,Z:\NOAA_AVHRR\NSS.GHRR.NL.D08306.S0135.E0321.B...,NOAA-16_tle.txt,2008-11-01,"[[-64.99261630859377, 85.23978322277314], [144...",2008-11-01 02:00:00
4,4,NSS.GHRR.NM.D08306.S0137.E0323.B3302830.WI,GHRR,NOAA-17,2008-11-01 01:37:00,2008-11-01 03:23:00,Z:\NOAA_AVHRR\NSS.GHRR.NM.D08306.S0137.E0323.B...,NOAA-17_tle.txt,2008-11-01,"[[-47.38378359658267, 74.70564711723532], [-11...",2008-11-01 02:02:00
...,...,...,...,...,...,...,...,...,...,...,...
2291,2291,NSS.GHRR.M2.D08335.S2155.E2337.B1098586.SV,GHRR,Metop-A,2008-11-30 21:55:00,2008-11-30 23:37:00,Z:\NOAA_AVHRR\NSS.GHRR.M2.D08335.S2155.E2337.B...,Metop-A_tle.txt,2008-11-01,"[[-19.355283862879425, 50.03218853644102], [-5...",2008-11-30 23:14:00
2292,2292,NSS.GHRR.NL.D08335.S2301.E0054.B4223233.GC,GHRR,NOAA-16,2008-11-30 23:01:00,2008-12-01 00:54:00,Z:\NOAA_AVHRR\NSS.GHRR.NL.D08335.S2301.E0054.B...,NOAA-16_tle.txt,2008-11-01,"[[-44.17775211677569, 84.27574360786984], [-17...",2008-11-30 23:21:00
2293,2293,NSS.GHRR.NN.D08335.S2301.E0049.B1819900.GC,GHRR,NOAA-18,2008-11-30 23:01:00,2008-12-01 00:49:00,Z:\NOAA_AVHRR\NSS.GHRR.NN.D08335.S2301.E0049.B...,NOAA-18_tle.txt,2008-11-01,"[[-77.00734183686967, 84.71099221502931], [129...",2008-11-30 23:26:00
2294,2294,NSS.GHRR.NM.D08335.S2239.E0022.B3345455.GC,GHRR,NOAA-17,2008-11-30 22:39:00,2008-12-01 00:22:00,Z:\NOAA_AVHRR\NSS.GHRR.NM.D08335.S2239.E0022.B...,NOAA-17_tle.txt,2008-11-01,"[[-1.8841533997657969, 66.95727514794018], [-5...",2008-11-30 23:28:00


为每个路径的开始和结束添加额外的两个点来观察演变

In [7]:
new = []
for idx in tqdm(ERA5_tracks.path_index.unique()):
    single_track =ERA5_tracks.loc[ERA5_tracks.path_index==idx].copy().reset_index(drop=True)
    single_track['relative'] = np.arange(len(single_track))
    start = pd.DataFrame(single_track.iloc[0]).T
    end = pd.DataFrame(single_track.iloc[-1]).T
    end_index = len(single_track)-1
    start_time,end_time = single_track.loc[0,'time'],single_track.loc[end_index,'time']
    cilent_columns = ['peak_value']
    start.loc[:,cilent_columns] = np.nan
    end.loc[:,cilent_columns] = np.nan
    #增添两个点
    start1,start2 = start.copy(),start.copy()
    end1,end2 = end.copy(),end.copy()
    start1.loc[0,'time'] = start_time-timedelta(hours=1)
    start2.loc[0,'time'] = start_time-timedelta(hours=2)
    start1.loc[0,'relative'] = -1000
    start2.loc[0,'relative'] = -999
    end1.loc[end_index,'time'] = end_time+timedelta(hours=1)
    end2.loc[end_index,'time'] = end_time+timedelta(hours=2)
    end1.loc[end_index,'relative'] = 999
    end2.loc[end_index,'relative'] = 1000
    new.append(pd.concat([start2,start1,single_track,end1,end2],axis=0))
new_all_tracks = pd.concat(new,axis=0)
ERA5_tracks = new_all_tracks.reset_index(drop=True)
ERA5_tracks['point_index'] = np.arange(len(ERA5_tracks))
ERA5_tracks

100%|██████████| 470/470 [00:01<00:00, 347.20it/s]


,time,lon,lat,peak_value,path_index,relative,point_index
0,2008-11-01 02:00:00,-37.25,66.0,NaN,14,-999,0
1,2008-11-01 03:00:00,-37.25,66.0,NaN,14,-1000,1
2,2008-11-01 04:00:00,-37.25,66.0,2.193079,14,0,2
3,2008-11-01 05:00:00,-34.75,65.75,3.07728,14,1,3
4,2008-11-01 06:00:00,-34.75,65.75,3.145358,14,2,4
...,...,...,...,...,...,...,...
10590,2008-11-30 21:00:00,-20.0,65.5,1.689195,934,4,10590
10591,2008-11-30 22:00:00,-19.75,65.25,1.537685,934,5,10591
10592,2008-11-30 23:00:00,-19.75,64.75,1.688163,934,6,10592
10593,2008-12-01 00:00:00,-19.75,64.75,NaN,934,999,10593


In [8]:
tle_dir = r'D:\Desktop\NOAA_TLE'
#波段4：红外
def read_AVHRR(filepath,tle_name):
    reader_cls = get_reader_class(filepath)
    reader = reader_cls(tle_dir=tle_dir, tle_name=tle_name)
    reader.read(filepath)
    channels = reader.get_calibrated_channels()
    lons, lats = reader.get_lonlat()
    IR = channels
    #异常扫描线去除-------------------------------------------------------------------------------
    masked_scanlines = np.isnan(IR[:,:,4]).sum(axis=1)>0
    IR = IR[~masked_scanlines]
    lons = lons[~masked_scanlines,:]
    lats = lats[~masked_scanlines,:]
    return IR,lons,lats

In [9]:
#匹配AVHRR
def judge_match(candidate_track,AVHRR_info):
    chosed_time,chosed_lon,chosed_lat = candidate_track.time,candidate_track.lon,candidate_track.lat
    #筛选在该小时内的所有AVHRR文件
    candidate_urlinfo = AVHRR_info.loc[np.logical_and(AVHRR_info.mean_time<chosed_time+timedelta(seconds=60*60),
                                        AVHRR_info.mean_time>chosed_time)]
    result = []
    for i in range(len(candidate_urlinfo)):
        basic_info =  candidate_urlinfo.iloc[i][['filepath','tle_name','border','dataset_type','satellite_name','mean_time']].copy()
        near_lon,near_lat,inside,min_dis = nearest_judge(chosed_lon,chosed_lat,basic_info.border)
        basic_info.border = json.dumps(basic_info.border.tolist())
        #在图像内部并且边缘距离大于400
        if inside&(min_dis>250):
            temp_track = pd.DataFrame(candidate_track).T
            temp_track[['filepath','tle_name','border','dataset_type','satellite_name','mean_time']] = basic_info.values
            result.append(temp_track)
    return result


粗匹配

In [13]:
result  = []
for idx in tqdm(range(len(ERA5_tracks)),desc='匹配中',position=1):
    candidate_track = ERA5_tracks.iloc[idx]
    temp = judge_match(ERA5_tracks.iloc[idx],AVHRR_info)
    result += temp
ERA5_matched = pd.concat(result,axis=0)
ERA5_matched


匹配中:   0%|          | 0/10595 [00:00<?, ?it/s]

匹配中: 100%|██████████| 10595/10595 [00:32<00:00, 326.28it/s]


,time,lon,lat,peak_value,path_index,relative,point_index,filepath,tle_name,border,dataset_type,satellite_name,mean_time
3,2008-11-01 05:00:00,-34.75,65.75,3.07728,14,1,3,Z:\NOAA_AVHRR\NSS.GHRR.NN.D08306.S0428.E0623.B...,NOAA-18_tle.txt,"[[163.44504880105987, 79.0408163248236], [69.1...",GHRR,NOAA-18,2008-11-01 05:18:00
5,2008-11-01 07:00:00,-34.75,65.75,2.937778,14,3,5,Z:\NOAA_AVHRR\NSS.GHRR.NL.D08306.S0650.E0844.B...,NOAA-16_tle.txt,"[[-149.56250954658867, 84.85938207827469], [69...",GHRR,NOAA-16,2008-11-01 07:37:00
6,2008-11-01 08:00:00,-34.75,65.75,2.486365,14,4,6,Z:\NOAA_AVHRR\NSS.GHRR.NK.D08306.S0741.E0931.B...,NOAA-15_tle.txt,"[[168.12049087889875, 81.49450779387746], [67....",GHRR,NOAA-15,2008-11-01 08:35:00
7,2008-11-01 09:00:00,-34.75,65.75,2.058764,14,5,7,Z:\NOAA_AVHRR\NSS.GHRR.NL.D08306.S0839.E1030.B...,NOAA-16_tle.txt,"[[114.54905833762683, 58.080957374128886], [68...",GHRR,NOAA-16,2008-11-01 09:36:00
10,2008-11-01 12:00:00,-35.0,65.75,NaN,14,1000,10,Z:\NOAA_AVHRR\NSS.LHRR.NM.D08306.S1212.E1223.B...,NOAA-17_tle.txt,"[[-87.01370491859014, 84.41902109085663], [42....",LHRR,NOAA-17,2008-11-01 12:17:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10586,2008-11-30 17:00:00,-20.75,66.75,1.980021,934,0,10586,Z:\NOAA_AVHRR\NSS.GHRR.NL.D08335.S1624.E1819.B...,NOAA-16_tle.txt,"[[-2.1461464109513897, 49.99595128402519], [-4...",GHRR,NOAA-16,2008-11-30 17:29:00
10588,2008-11-30 19:00:00,-19.5,66.5,1.85562,934,2,10588,Z:\NOAA_AVHRR\NSS.GHRR.M2.D08335.S1831.E2013.B...,Metop-A_tle.txt,"[[31.324650594674818, 50.03721590440673], [-4....",GHRR,Metop-A,2008-11-30 19:45:00
10589,2008-11-30 20:00:00,-20.25,65.75,1.747692,934,3,10589,Z:\NOAA_AVHRR\NSS.GHRR.NM.D08335.S1918.E2102.B...,NOAA-17_tle.txt,"[[21.62490908704601, 50.00615332055338], [-14....",GHRR,NOAA-17,2008-11-30 20:21:00
10590,2008-11-30 21:00:00,-20.0,65.5,1.689195,934,4,10590,Z:\NOAA_AVHRR\NSS.GHRR.M2.D08335.S2012.E2155.B...,Metop-A_tle.txt,"[[5.9840342981091945, 50.034388536532404], [-3...",GHRR,Metop-A,2008-11-30 21:26:00


根据涡度位置重匹配

In [18]:
def calculate_meantime(filepath,tle_name,central_lon,central_lat):
    #计算中心点100KM内的平均扫描时间
    tle_dir = r'Z:\NOAA_TLE'
    reader_cls = get_reader_class(filepath)
    reader = reader_cls(tle_dir=tle_dir, tle_name=tle_name)
    reader.read(filepath)
    times = reader.get_times()
    lons, lats = reader.get_lonlat()
    dis_array = geodistance(central_lon,central_lat,lons,lats)
    scantime150 = times[(dis_array<100).sum(axis=1)>0]
    #去除空值
    scantime150 = scantime150[~np.isnan(scantime150)]
    if len(scantime150)==0:
        return np.nan
    meantime = pd.Series(scantime150).mean()
    #规整到分钟
    return pd.to_datetime( meantime.strftime('%Y%m%d_%H%M'),format='%Y%m%d_%H%M')

In [ ]:
unique_path_indexs = ERA5_matched.path_index.unique()
result = []
for i in tqdm(range(len(unique_path_indexs))):
    chosed_ERA5_matched = ERA5_matched.loc[ERA5_matched.path_index==unique_path_indexs[i]].copy()
    chosed_ERA5_track = ERA5_tracks.loc[ERA5_tracks.path_index==unique_path_indexs[i]].copy().reset_index(drop=True)
    chosed_ERA5_track['time'] = chosed_ERA5_track['time'].astype('datetime64[ns]')
    #重计算扫描到涡度中心时的时间
    meantimes = []
    for i in range(len(chosed_ERA5_matched)):
        chosed_cloud_info = chosed_ERA5_matched.iloc[i]
        meantimes.append(calculate_meantime(chosed_cloud_info.filepath,chosed_cloud_info.tle_name,chosed_cloud_info.lon,chosed_cloud_info.lat))
    chosed_ERA5_matched['mean_time'] = meantimes 
    old_AVHRR_info = chosed_ERA5_matched[['lon','lat','filepath', 'tle_name','dataset_type', 'satellite_name','mean_time']].copy()\
                                .rename(columns={'lon':'old_lon','lat':'old_lat'})
    old_AVHRR_info['time'] = old_AVHRR_info['mean_time'].dt.floor('h')
    result.append(pd.merge(chosed_ERA5_track,old_AVHRR_info,on='time',how='inner'))
new_ERA5_matched = pd.concat(result)
new_ERA5_matched

100%|██████████| 469/469 [1:14:55<00:00,  9.59s/it]


,index,time,lon,lat,peak_value,path_index,relative,point_index,old_lon,old_lat,filepath,tle_name,dataset_type,satellite_name,mean_time
0,2,2008-11-01 04:00:00,-37.25,66.0,2.193079,14,0,2,-34.75,65.75,Z:\NOAA_AVHRR\NSS.GHRR.NN.D08306.S0428.E0623.B...,NOAA-18_tle.txt,GHRR,NOAA-18,2008-11-01 04:46:00
1,5,2008-11-01 07:00:00,-34.75,65.75,2.937778,14,3,5,-34.75,65.75,Z:\NOAA_AVHRR\NSS.GHRR.NL.D08306.S0650.E0844.B...,NOAA-16_tle.txt,GHRR,NOAA-16,2008-11-01 07:07:00
2,6,2008-11-01 08:00:00,-34.75,65.75,2.486365,14,4,6,-34.75,65.75,Z:\NOAA_AVHRR\NSS.GHRR.NK.D08306.S0741.E0931.B...,NOAA-15_tle.txt,GHRR,NOAA-15,2008-11-01 08:39:00
3,7,2008-11-01 09:00:00,-34.75,65.75,2.058764,14,5,7,-34.75,65.75,Z:\NOAA_AVHRR\NSS.GHRR.NL.D08306.S0839.E1030.B...,NOAA-16_tle.txt,GHRR,NOAA-16,2008-11-01 09:39:00
4,9,2008-11-01 11:00:00,-35.0,65.75,NaN,14,999,9,-35.0,65.75,Z:\NOAA_AVHRR\NSS.GHRR.M2.D08306.S1150.E1332.B...,Metop-A_tle.txt,GHRR,Metop-A,2008-11-01 11:55:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,10586,2008-11-30 17:00:00,-20.75,66.75,1.980021,934,0,10586,-20.75,66.75,Z:\NOAA_AVHRR\NSS.GHRR.NL.D08335.S1624.E1819.B...,NOAA-16_tle.txt,GHRR,NOAA-16,2008-11-30 17:17:00
4,10589,2008-11-30 20:00:00,-20.25,65.75,1.747692,934,3,10589,-19.5,66.5,Z:\NOAA_AVHRR\NSS.GHRR.M2.D08335.S1831.E2013.B...,Metop-A_tle.txt,GHRR,Metop-A,2008-11-30 20:10:00
5,10589,2008-11-30 20:00:00,-20.25,65.75,1.747692,934,3,10589,-20.25,65.75,Z:\NOAA_AVHRR\NSS.GHRR.NM.D08335.S1918.E2102.B...,NOAA-17_tle.txt,GHRR,NOAA-17,2008-11-30 20:08:00
6,10590,2008-11-30 21:00:00,-20.0,65.5,1.689195,934,4,10590,-20.0,65.5,Z:\NOAA_AVHRR\NSS.GHRR.M2.D08335.S2012.E2155.B...,Metop-A_tle.txt,GHRR,Metop-A,2008-11-30 21:50:00


命名

In [ ]:
#记录point_index出现的次数,主要是用来统计一个小小时内多次出现的点
unique_point_index = new_ERA5_matched.drop_duplicates(subset=['point_index'],keep='first').point_index
point_index_count = new_ERA5_matched.value_counts('point_index').to_frame().loc[unique_point_index]
counts = [list(range(i)) for i in point_index_count.loc[:,'count'].values]
sum_counts = sum(counts,[])

In [22]:
path_indexs = new_ERA5_matched.path_index.values
point_indexs = new_ERA5_matched.point_index.values
nums = np.array(sum_counts)
new_filenames = []
for i in tqdm(range(len(nums))):
    if nums[i]==0:
        filename = 'path%d_%d.png'%(path_indexs[i],point_indexs[i])
    else:
        filename = 'path%d_%d_%d.png'%(path_indexs[i],point_indexs[i],nums[i])
    new_filenames.append(filename)
new_ERA5_matched['filename'] = new_filenames
new_ERA5_matched = new_ERA5_matched.infer_objects()

100%|██████████| 8434/8434 [00:00<00:00, 830023.23it/s]


In [24]:
new_ERA5_matched.drop(columns='index').to_excel(r"D:\Desktop\pycode\PLs_identity\IMPMCT\example_data\matched_ERA5_tracks_2008_11.xlsx",index=False)